In [12]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from playwright.async_api import async_playwright
import pandas as pd
from tqdm.asyncio import tqdm_asyncio

# List of ZIP codes to scrape
zip_codes = [
    "77002","77021","77040","77059","77078","77098","77384","77479","77547",
    "77003","77022","77041","77060","77079","77099","77385","77484","77571",
    "77004","77023","77042","77061","77080","77204","77386","77489","77573",
    "77005","77024","77043","77062","77081","77325","77388","77492","77574",
    "77006","77025","77044","77063","77082","77336","77389","77493","77578",
    "77007","77026","77045","77064","77083","77338","77396","77494","77581",
    "77008","77027","77046","77065","77084","77339","77401","77497","77584",
    "77009","77028","77047","77066","77085","77345","77406","77498","77586",
    "77010","77029","77048","77067","77086","77346","77407","77502","77587",
    "77011","77030","77049","77068","77087","77354","77429","77503","77588",
    "77012","77031","77050","77069","77088","77357","77433","77504","77598",
    "77013","77032","77051","77070","77089","77365","77447","77505","77014",
    "77033","77052","77071","77090","77373","77449","77506","77015","77034",
    "77053","77072","77091","77375","77450","77507","77016","77035","77054",
    "77073","77092","77377","77459","77520","77017","77036","77055","77074",
    "77093","77379","77469","77530","77018","77037","77056","77075","77094",
    "77380","77472","77532","77019","77038","77057","77076","77095","77381",
    "77477","77536","77020","77039","77058","77077","77096","77382","77478",
    "77546"
]

async def scrape_zip(zip_code, headless=True):
    async with async_playwright() as p:
        browser = await p.firefox.launch(headless=headless)
        page = await browser.new_page()
        await page.goto("https://www.tdlr.texas.gov/tools_search/", timeout=60000)

        # Click ZIP radio button to enable ZIP input
        await page.click('#zipcodebutton')
        await page.fill('#zipcodedata', zip_code)

        # Select Tow Truck and Active status
        await page.select_option('#zip_carrier_type', 'tow')
        await page.select_option('#zip_status', 'A')

        # Click the Search button
        await page.click('#submit3')

        # Wait for table or "No results" message
        try:
            await page.wait_for_selector('table[bordercolor="black"]', timeout=10000)
            rows = await page.query_selector_all('table[bordercolor="black"] tr')
            results = []
            for row in rows[1:]:  # skip header
                cells = await row.query_selector_all('td')
                if len(cells) >= 6:
                    results.append([await cell.inner_text() for cell in cells[:6]])
            if not results:
                results = [[zip_code, "No results", "", "", "", ""]]
        except:
            # No table found, return "No results"
            results = [[zip_code, "No results", "", "", "", ""]]

        await browser.close()
        return results

async def scrape_all_zips():
    all_results = []
    for z in tqdm_asyncio(zip_codes, desc="Scraping ZIP codes"):
        try:
            data = await scrape_zip(z)
            all_results.extend(data)
        except Exception as e:
            all_results.append([z, f"Error: {e}", "", "", "", ""])
    df = pd.DataFrame(all_results, columns=['Customer', 'DBA Name', 'TDLR Number', 'City', 'State', 'Zip code'])
    return df

# Run in Jupyter
loop = asyncio.get_event_loop()
all_data = loop.run_until_complete(scrape_all_zips())

# Save to CSV
all_data.to_csv("tow_truck_licenses.csv", index=False)
print(all_data)



Scraping ZIP codes: 100%|██████████| 163/163 [10:18<00:00,  3.80s/it]

                         Customer                   DBA Name TDLR Number  \
0                           77002                 No results               
1           A&R BRAKE SERVICE LLC  KAR CARRIEGE TRANSMISSION  006583164C   
2          IMPERIAL TRANSPORT LLC            IMPERIAL TOWING  006552449C   
3    J&N TOWING AND TRANSPORT LLC                             006570303C   
4    MARY'S TOWING & RECOVERY INC                             006622335C   
..                            ...                        ...         ...   
687               AL-AMANA TOWING                             006407827C   
688   PRO WRECKER ENTERPRISES LLC         ALFA AUTO SERVICES  006506392C   
689             E & E TOWING, LLC                             006582043C   
690                FERNANDO GARZA  MTY RECOVERY AND SERVICES  006524304C   
691                  LUIS GUEVARA             GUEVARA TOWING  006449697C   

            City State Zip code  
0                                
1        HOUSTON   